# Inicialização

In [1]:
import numpy as np
import pandas as pd
import ifcopenshell
import ifcopenshell.util.element
import ifcopenshell.util.selector
import ifcopenshell.api.sequence

Note: API not available due to missing dependencies: geometry.add_door_representation - No module named 'mathutils'
Note: API not available due to missing dependencies: geometry.add_railing_representation - No module named 'mathutils'
Note: API not available due to missing dependencies: geometry.add_representation - No module named 'bpy'
Note: API not available due to missing dependencies: geometry.add_window_representation - No module named 'mathutils'
Note: API not available due to missing dependencies: grid.create_axis_curve - No module named 'mathutils'
Note: API not available due to missing dependencies: sequence.recalculate_schedule - No module named 'networkx'


In [2]:
### pendencias
# 1. Verificar como separar no QiBuilder o que está em prumada, aéreo, embutido em parede
# 2. Categorizar os itens entre prumada, ramal, subcoletor, reservação, etc.
######

# Funções

## Gera combinações

In [3]:
# gera combinações para criar a EAP
def criar_combinacoes(niveis, *args):
    """Cria combinações de níveis, com o objetivo de gerar EAP e descrição do item da EAP."""
    # Cria um DataFrame auxiliar com todas as combinações
    combinacoes = pd.MultiIndex.from_product(args, names=niveis).to_frame(index=False)

    # Gera a coluna EAP
    combinacoes["EAP"] = (
        combinacoes.apply(lambda row: SEPARADOR_EAP.join([str(np.where(arg == row.iloc[i])[0][0] + 1) for i, arg in enumerate (args)]
                                                        ), axis=1)
    )

    # Gera a descrição
    combinacoes["descrição"] = combinacoes.loc[:, combinacoes.columns != "EAP"].apply(lambda row: SEPARADOR_DESCRICAO.join(row.astype(str)), axis=1)
    
    return combinacoes

## Incrementa EAP

In [4]:
# Função para incrementar o valor de uma string de EAP
def incrementar_eap(string, posicao, incremento):
    partes = string.split(".") # divide a string em partes
    partes[posicao] = str(int(partes[posicao]) + incremento) # incrementa o elemento na posição
    return ".".join(partes) # junta novamente a string

# IFC

## IFC QiBuilder

In [209]:
FILE_PATH = "./modelo/FERNANDA E FILIPE-PE-HID-IFC.ifc"
model = ifcopenshell.open(FILE_PATH)

In [210]:
# Classes IFC
classes_unicas = set()
for entidade in model:
    classes_unicas.add(entidade.is_a())

In [211]:
# dicionário para armazenar as quantidades dos elementos
dict_quantitativo = {
    "pavimento": [],
    "rede": [],
    "item": [],
    "quantidade": [],
    "ID": [],
}
# pega as classes IFC que tem no arquivo
for classe in classes_unicas:
    elementos = model.by_type(classe)
    for elemento in elementos:
        # propriedades e quantidades da classe IFC
        psets_and_qtos = ifcopenshell.util.element.get_psets(elemento)
        
        # como o IFC é do QiBuilder, ele exporta em um conjunto específico
        if 'AltoQi_QiBuilder-Itens_Associados' in psets_and_qtos: 
            for item, quantidade in psets_and_qtos['AltoQi_QiBuilder-Itens_Associados'].items():
                if item != "id": # remove o id
                    pavimento = ifcopenshell.util.element.get_container(elemento)[2]
                    rede = psets_and_qtos["Identificação_Elemento"]['Rede']
                    
                    dict_quantitativo["pavimento"].append(pavimento)
                    dict_quantitativo["rede"].append(rede)
                    dict_quantitativo["item"].append(item)
                    dict_quantitativo["quantidade"].append(quantidade)
                    dict_quantitativo["ID"].append(elemento.GlobalId)

# cria o dataframe no pandas
df_quantitativo = pd.DataFrame(dict_quantitativo)

In [212]:
df_quantitativo

,pavimento,rede,item,quantidade,ID
0,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R
1,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,23vghV9i5E69JyH8cCXCzC
2,TER NA,Alimentação (Piscina-FIL),"Metais - Valvula de retenção vertical - 1.1/2""",1.0,3Xe7iW2cXEUfq5MHSwOCQP
3,TER NA,Alimentação (Piscina-FIL),PVC rígido soldável - Adapt sold.curto c/bolsa...,2.0,3Xe7iW2cXEUfq5MHSwOCQP
4,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,3mVMY5yAD9uAhX6Sme9ohP
...,...,...,...,...,...
552,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,1K8ErAvxDEhuVB3L0$rilu
553,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,2gUdsZucfD09_ho9YhCYi8
554,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,1HEBKtuuD6PhkNQxhOCT7h
555,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,2c8JUCCDjFr8EWB5NyNC1Q


In [ ]:
# quantitativo agrupado para conferência
df_quantitativo_agrupado = df_quantitativo.groupby(["pavimento", "rede", "item"], as_index=False)["quantidade"].sum()

In [ ]:
df_quantitativo_agrupado

In [ ]:
df_quantitativo_agrupado.to_excel("lista de materiais.xlsx")

## IFC TQS-Revit-IFC

In [5]:
FILE_PATH = "./modelo/789-EST-EX-001-MOD-GER-XX-MODELO_3D_GERAL-R01-IFC4.ifc"
model = ifcopenshell.open(FILE_PATH)

In [6]:
# Classes IFC
classes_unicas = set()
for entidade in model:
    classes_unicas.add(entidade.is_a())

In [7]:
# dicionário para armazenar as quantidades dos elementos
dict_quantitativo = {
    "pavimento": [],
    "junta": [],
    "elemento": [],
    "tipo": [],
    "codigo": [],
    "comprimento (cm)": [],
    "largura (cm)": [],
    "altura (cm)": [],
    "área (m²)": [],
    "volume (m³)": [],
    "GlobalID": [],
}

# remove as classes tipos e pavimento
classe_a_remover = ["IfcBeamType", "IfcColumnType", "IfcBuildingStorey"]
classes_unicas = [classe for classe in classes_unicas if classe not in classe_a_remover]

# função para adicionar os elementos no dicionário
def add_element_to_dict(pavimento, junta, elemento, tipo, codigo, comprimento, largura, altura, area, volume, global_id):
    dict_quantitativo["pavimento"].append(pavimento)
    dict_quantitativo["junta"].append(junta)
    dict_quantitativo["elemento"].append(elemento)
    dict_quantitativo["tipo"].append(tipo)
    dict_quantitativo["codigo"].append(codigo)
    dict_quantitativo["comprimento (cm)"].append(comprimento)
    dict_quantitativo["largura (cm)"].append(largura)
    dict_quantitativo["altura (cm)"].append(altura)
    dict_quantitativo["área (m²)"].append(area)
    dict_quantitativo["volume (m³)"].append(volume)
    dict_quantitativo["GlobalID"].append(global_id)

# mapemaneto dos parâmetros de cada uma das classes para o dicionário
class_handlers = {
    "IfcSlab": lambda psets_and_qtos, pavimento, junta: (
        ("Cortina" if psets_and_qtos["Dados de identidade"]["Titulo"] == "CORTINA" else psets_and_qtos["Dados de identidade"]["Tipo Elemento"],
         "" if psets_and_qtos["Dados de identidade"]["Titulo"] == "CORTINA" else psets_and_qtos["Dados"]["Tipo"],
         psets_and_qtos["Dados de identidade"]["Titulo"],
         "",
         "",
         "" if psets_and_qtos["Dados de identidade"]["Titulo"] == 'CORTINA' or psets_and_qtos["Dados"]["Tipo"] == "Nerv T" else psets_and_qtos["Dados"]["Altura_cm"],
         psets_and_qtos["Qto_SlabBaseQuantities"]["NetArea"],
         psets_and_qtos["Qto_SlabBaseQuantities"]["NetVolume"])
    ),
    "IfcBeam": lambda psets_and_qtos, pavimento, junta: (
        (psets_and_qtos["Dados de identidade"]["Tipo Elemento"],
         "",
         psets_and_qtos["Dados de identidade"]["Titulo"],
         psets_and_qtos["Qto_BeamBaseQuantities"]["Length"],
         psets_and_qtos["Dados"]["Largura_cm"],
         psets_and_qtos["Dados"]["Altura_cm"],
         "",
         psets_and_qtos["Qto_BeamBaseQuantities"]["NetVolume"])
    ),
    "IfcColumn": lambda psets_and_qtos, pavimento, junta: (
        (psets_and_qtos["Dados de identidade"]["Tipo Elemento"],
         "",
         psets_and_qtos["Dados de identidade"]["Titulo"],
         psets_and_qtos["Qto_ColumnBaseQuantities"]["Length"],
         psets_and_qtos["Dados"]["Largura_cm"] if psets_and_qtos["Dados"]["Secao"].startswith("R") else psets_and_qtos["Pset_QuantityTakeOff"]["Reference"],
         psets_and_qtos["Dados"]["Altura_cm"] if psets_and_qtos["Dados"]["Secao"].startswith("R") else "",
         psets_and_qtos["Qto_ColumnBaseQuantities"]["OuterSurfaceArea"],
         psets_and_qtos["Qto_ColumnBaseQuantities"]["NetVolume"])
    ),
    "IfcWallStandardCase": lambda psets_and_qtos, pavimento, junta: (
        ("Cortina",
         "",
         "",
         psets_and_qtos["Qto_WallBaseQuantities"]["Length"],
         psets_and_qtos["Qto_WallBaseQuantities"]["Width"],
         psets_and_qtos["Qto_WallBaseQuantities"]["Height"],
         psets_and_qtos["Qto_WallBaseQuantities"]["NetSideArea"],
         psets_and_qtos["Qto_WallBaseQuantities"]["NetVolume"])
    ),
}

# Processa os elementos
for classe in classes_unicas:
    elementos = model.by_type(classe)
    for elemento in elementos:
        psets_and_qtos = ifcopenshell.util.element.get_psets(elemento)
        if 'Cotas' in psets_and_qtos and 'Volume' in psets_and_qtos["Cotas"]:
            pavimento = ifcopenshell.util.element.get_container(elemento)[2]
            junta = psets_and_qtos["My_Data"]["Junta"]
            handler = class_handlers.get(classe)

            if handler:
                print(elemento.GlobalId)
                print()
                results = handler(psets_and_qtos, pavimento, junta)
                add_element_to_dict(pavimento, junta, *results, elemento.GlobalId)

            else:
                print(f"Classe não considerada: {classe}, GlobalId: {elemento.GlobalId}")

        elif 'Cotas' in psets_and_qtos:
            print(f"Erro de modelagem, GlobalId: {elemento.GlobalId}")

df_quantitativo = pd.DataFrame(dict_quantitativo)

0$TvfvQWT3UPXp$fNe6P3J

0$TvfvQWT3UPXp$fNe6P2Q

0$TvfvQWT3UPXp$fNe6P0F

0$TvfvQWT3UPXp$fNe6PV9

0$TvfvQWT3UPXp$fNe6Ond

0$TvfvQWT3UPXp$fNe6P1J

0$TvfvQWT3UPXp$fNe6OpT

0vkyWjqLn1CxlLegVyhPKA

0vkyWjqLn1CxlLegVyhPK5

0vkyWjqLn1CxlLegVyhPKP

0vkyWjqLn1CxlLegVyhPKK

0vkyWjqLn1CxlLegVyhPLw

0vkyWjqLn1CxlLegVyhPLF

0vkyWjqLn1CxlLegVyhPLq

0vkyWjqLn1CxlLegVyhOtu

0vkyWjqLn1CxlLegVyhOtz

0vkyWjqLn1CxlLegVyhPL7

0vkyWjqLn1CxlLegVyhPMW

0vkyWjqLn1CxlLegVyhPM9

0vkyWjqLn1CxlLegVyhPNi

0vkyWjqLn1CxlLegVyhPN3

0vkyWjqLn1CxlLegVyhPG5

0vkyWjqLn1CxlLegVyhPGT

0vkyWjqLn1CxlLegVyhPGP

0vkyWjqLn1CxlLegVyhPHC

0vkyWjqLn1CxlLegVyhPHY

0vkyWjqLn1CxlLegVyhPNB

0vkyWjqLn1CxlLegVyhPNy

0vkyWjqLn1CxlLegVyhPHV

0vkyWjqLn1CxlLegVyhPIh

0vkyWjqLn1CxlLegVyhPHG

0vkyWjqLn1CxlLegVyhPHL

0vkyWjqLn1CxlLegVyhPHi

0vkyWjqLn1CxlLegVyhPNI

0vkyWjqLn1CxlLegVyhPNM

0vkyWjqLn1CxlLegVyhPNQ

0vkyWjqLn1CxlLegVyhPNa

0vkyWjqLn1CxlLegVyhPLa

0vkyWjqLn1CxlLegVyhPLe

0vkyWjqLn1CxlLegVyhPLi

0vkyWjqLn1CxlLegVyhPKG

0vkyWjqLn1CxlLeg

In [8]:
df_quantitativo

,pavimento,junta,elemento,tipo,codigo,comprimento (cm),largura (cm),altura (cm),área (m²),volume (m³),GlobalID
0,Fundacao,Junta D,Cortina,,,522.421316,25.0,480.0,25.076223,6.269056,0$TvfvQWT3UPXp$fNe6P3J
1,Fundacao,Junta D,Cortina,,,1511.5,19.0,480.0,72.552,13.784880,0$TvfvQWT3UPXp$fNe6P2Q
2,Fundacao,Junta D,Cortina,,,709.0,19.0,480.0,34.032,6.466080,0$TvfvQWT3UPXp$fNe6P0F
3,Fundacao,Junta D,Cortina,,,3301.004362,19.0,480.0,158.448209,30.105160,0$TvfvQWT3UPXp$fNe6PV9
4,Fundacao,Junta B,Cortina,,,5002.997283,19.0,480.0,240.14387,45.627335,0$TvfvQWT3UPXp$fNe6Ond
...,...,...,...,...,...,...,...,...,...,...,...
1621,Fundacao,Junta F,Pilar,,P606,60.0,19.0,60.0,0.632,0.045600,2dwaBY7dL8M8AzW4i4MGM5
1622,Fundacao,Junta F,Pilar,,P602,60.0,60.0,19.0,0.632,0.045600,2dwaBY7dL8M8AzW4i4MGM3
1623,Fundacao,Junta F,Pilar,,P605,60.0,60.0,19.0,0.632,0.045600,2dwaBY7dL8M8AzW4i4MGM1
1624,Fundacao,Junta F,Pilar,,P603,60.0,60.0,19.0,0.632,0.045600,2dwaBY7dL8M8AzW4i4MGM$


In [9]:
# todos os elementos não descritos abaixo terão a área de forma igual a área calculada pelo IFC
df_quantitativo["área de forma (m²)"] = df_quantitativo["área (m²)"]

elementos = df_quantitativo["elemento"].unique()
for elemento in elementos:
    # quando cortina, multiplica por 2 para ter a área das 2 faces da parede
    if elemento == "Cortina":
        df_quantitativo.loc[df_quantitativo["elemento"] == elemento, "área de forma (m²)"] = df_quantitativo.loc[df_quantitativo["elemento"] == elemento, "área de forma (m²)"] * 2
    # quando é laje em solo não tem forma
    elif elemento == "Laje":
        df_quantitativo.loc[(df_quantitativo["pavimento"] == "Fundacao") & (df_quantitativo["elemento"] == "Laje"), "área de forma (m²)"] = 0
    # quando viga, multiplica os lados para ter a área de forma estimada
    elif elemento == "Viga":
        viga_mask = df_quantitativo["elemento"] == "Viga"
        L = df_quantitativo.loc[viga_mask, "comprimento (cm)"] / 100
        B = df_quantitativo.loc[viga_mask, "largura (cm)"] / 100
        H = df_quantitativo.loc[viga_mask, "altura (cm)"] / 100
        df_quantitativo.loc[viga_mask, "área de forma (m²)"] = L * B + L * H * 2        

In [10]:
df_quantitativo.to_excel("EST-lista de materiais.xlsx")

## IFC REVIT - ARQ

In [77]:
FILE_PATH = "./modelo/789-ARQ-PE-XXX-MOD-GER-XX-MODELO_IFC_____-R13-IFC4.ifc"
model = ifcopenshell.open(FILE_PATH)

In [3]:
# Classes IFC
classes_unicas = set()
for entidade in model:
    classes_unicas.add(entidade.is_a())

In [4]:
classes_unicas

{'IfcActorRole',
 'IfcApplication',
 'IfcArbitraryClosedProfileDef',
 'IfcArbitraryProfileDefWithVoids',
 'IfcAxis2Placement2D',
 'IfcAxis2Placement3D',
 'IfcBeam',
 'IfcBeamType',
 'IfcBooleanClippingResult',
 'IfcBuilding',
 'IfcBuildingElementProxy',
 'IfcBuildingElementProxyType',
 'IfcBuildingStorey',
 'IfcCartesianPoint',
 'IfcCartesianTransformationOperator3D',
 'IfcCircle',
 'IfcCircleProfileDef',
 'IfcClassification',
 'IfcClassificationReference',
 'IfcClosedShell',
 'IfcColourRgb',
 'IfcColumn',
 'IfcColumnType',
 'IfcCompositeCurve',
 'IfcCompositeCurveSegment',
 'IfcConnectedFaceSet',
 'IfcConversionBasedUnit',
 'IfcCovering',
 'IfcCoveringType',
 'IfcCurtainWall',
 'IfcCurtainWallType',
 'IfcDerivedUnit',
 'IfcDerivedUnitElement',
 'IfcDimensionalExponents',
 'IfcDirection',
 'IfcDistributionPort',
 'IfcDoor',
 'IfcDoorLiningProperties',
 'IfcDoorPanelProperties',
 'IfcDoorStyle',
 'IfcElementAssembly',
 'IfcElementQuantity',
 'IfcExtrudedAreaSolid',
 'IfcFace',
 'IfcFace

In [78]:
elemento = model.by_type("IfcWall")[0]

In [79]:
elemento.ObjectType

'Parede básica:ACABAMENTO_SECO_20mm_01'

In [80]:
ifcopenshell.util.element.get_psets(elemento)

{'Pset_QuantityTakeOff': {'Reference': 'ACABAMENTO_SECO_20mm_01',
  'id': 1391908},
 'Pset_ReinforcementBarPitchOfWall': {'Reference': 'ACABAMENTO_SECO_20mm_01',
  'id': 1420018},
 'Pset_WallCommon': {'IsExternal': False,
  'Reference': 'ACABAMENTO_SECO_20mm_01',
  'id': 1391906,
  'LoadBearing': False,
  'ExtendToStructure': False},
 'Qto_WallBaseQuantities': {'GrossSideArea': 4.38900000000004,
  'GrossVolume': 0.0877800000000188,
  'Height': 3.85,
  'Length': 1.14000000000001,
  'NetSideArea': 4.21800000000004,
  'NetVolume': 0.0843600000000181,
  'Width': 0.0200000000000041,
  'id': 1656143}}

In [91]:
# dicionário para armazenar as quantidades dos elementos
dict_quantitativo = {
    "classe": [],
    "pavimento": [],
    "junta": [],
    "elemento": [],
    "codigo": [],
    "comprimento (m)": [],
    "largura (m)": [],
    "altura (m)": [],
    "perimetro (m)": [],
    "área (m²)": [],
    "volume (m³)": [],
    "GlobalID": [],
}

# função para adicionar os elementos no dicionário
def add_element_to_dict(classe, pavimento, junta, elemento, codigo, comprimento, largura, altura, perimetro, area, volume, global_id):
    dict_quantitativo["classe"].append(classe)
    dict_quantitativo["pavimento"].append(pavimento)
    dict_quantitativo["junta"].append(junta)
    dict_quantitativo["elemento"].append(elemento)
    dict_quantitativo["codigo"].append(codigo)
    dict_quantitativo["comprimento (m)"].append(comprimento)
    dict_quantitativo["largura (m)"].append(largura)
    dict_quantitativo["altura (m)"].append(altura)
    dict_quantitativo["perimetro (m)"].append(perimetro)
    dict_quantitativo["área (m²)"].append(area)
    dict_quantitativo["volume (m³)"].append(volume)
    dict_quantitativo["GlobalID"].append(global_id)

# mapemaneto dos parâmetros de cada uma das classes para o dicionário
class_handlers = {
    "IfcWall": lambda psets_and_qtos, classe, pavimento, junta: (
        (psets_and_qtos["Pset_QuantityTakeOff"]["Reference"],
         "",
         psets_and_qtos["Qto_WallBaseQuantities"]["Length"],
         psets_and_qtos["Qto_WallBaseQuantities"]["Width"],
         psets_and_qtos["Qto_WallBaseQuantities"]["Height"],
         "",
         psets_and_qtos["Qto_WallBaseQuantities"]["NetSideArea"],
         psets_and_qtos["Qto_WallBaseQuantities"]["NetVolume"])
    ),
    "IfcWallStandardCase": lambda psets_and_qtos, classe, pavimento, junta: (
        (psets_and_qtos["Pset_QuantityTakeOff"]["Reference"],
         "",
         psets_and_qtos["Qto_WallBaseQuantities"]["Length"],
         psets_and_qtos["Qto_WallBaseQuantities"]["Width"],
         psets_and_qtos["Qto_WallBaseQuantities"]["Height"],
         "",
         psets_and_qtos["Qto_WallBaseQuantities"]["NetSideArea"],
         psets_and_qtos["Qto_WallBaseQuantities"]["NetVolume"])
    ),
    "IfcSlab": lambda psets_and_qtos, classe, pavimento, junta: (
        (psets_and_qtos["Pset_QuantityTakeOff"]["Reference"],
         "",
         "",
         "",
         psets_and_qtos["Qto_SlabBaseQuantities"]["Depth"],
         psets_and_qtos["Qto_SlabBaseQuantities"]["Perimeter"],         
         psets_and_qtos["Qto_SlabBaseQuantities"]["NetArea"],
         psets_and_qtos["Qto_SlabBaseQuantities"]["NetVolume"])
    ),        
}

# Processa os elementos
elementos = ifcopenshell.util.selector.filter_elements(model, "IfcElement")
for elemento in elementos:
    if ifcopenshell.util.element.get_container(elemento) and ifcopenshell.util.element.get_psets(elemento, qtos_only=True):
        classe = elemento.is_a()
        psets_and_qtos = ifcopenshell.util.element.get_psets(elemento)
        pavimento = ifcopenshell.util.element.get_container(elemento)[2]
        junta = "GERAL"
        handler = class_handlers.get(classe)
    
        if handler:
            results = handler(psets_and_qtos, classe, pavimento, junta)
            add_element_to_dict(classe, pavimento, junta, *results, elemento.GlobalId)
    
        else:
            print(f"Classe não considerada: {classe}, GlobalId: {elemento.GlobalId}")

df_quantitativo = pd.DataFrame(dict_quantitativo)

Classe não considerada: IfcMember, GlobalId: 1joGBEqi5DvQ8CN9wcX$nv
Classe não considerada: IfcMember, GlobalId: 3HFgCWMzbFRA$y54Nhv_EQ
Classe não considerada: IfcMember, GlobalId: 249YOfh35Fwhe4TbOrg$ja
Classe não considerada: IfcMember, GlobalId: 0WUFfBF_5D1gigi2a8fIFG
Classe não considerada: IfcMember, GlobalId: 0LYqlXswT9mB8LHQL_yKww
Classe não considerada: IfcPlate, GlobalId: 3HFgCWMzbFRA$y54Nhv_2M
Classe não considerada: IfcMember, GlobalId: 2G9oBZ7vr3NP5_kKNa$ee2
Classe não considerada: IfcMember, GlobalId: 3HFgCWMzbFRA$y54Nhv_7t
Classe não considerada: IfcMember, GlobalId: 1ejmjhRo17Xe0Zzy8LpHWP
Classe não considerada: IfcMember, GlobalId: 3FqrN3TdrBxOShHBqONX$i
Classe não considerada: IfcMember, GlobalId: 0UFhjKWVn2RRSWFvbZHmV_
Classe não considerada: IfcMember, GlobalId: 2abrad_yTAk8FTs4SqnmqI
Classe não considerada: IfcMember, GlobalId: 2abrad_yTAk8FTs4Sqnm7d
Classe não considerada: IfcMember, GlobalId: 3HFgCWMzbFRA$y54Nhv_7c
Classe não considerada: IfcMember, GlobalId: 07Jw

In [92]:
df_quantitativo = pd.DataFrame(dict_quantitativo)

In [93]:
df_quantitativo.describe()

,altura (m),área (m²),volume (m³)
count,2404.000000,2404.000000,2404.000000
mean,2.644414,42.406989,6.903952
std,1.441678,440.285675,93.902252
min,0.004000,0.001171,0.000031
25%,1.800000,2.289000,0.133591
50%,3.030000,6.266897,0.424871
75%,3.520000,14.175000,1.030281
max,17.740000,16705.498889,3341.099778


In [94]:
df_quantitativo.to_excel("ARQ-lista de materiais-Paredes e Pisos.xlsx")

# SINAPI

## Acesso ao arquivo

In [55]:
# Acessa o arquivo
SUFIXO = "SINAPI_ref_Insumos_Composicoes"
estado = "DF"
ano = "2024"
mes = "09"
tipo = "Desonerado"
SUFIXO_ARQ = "SINAPI_Custo_Ref_Composicoes_Analitico"
#ARQUIVO = "SINAPI_Custo_Ref_Composicoes_Analitico_DF_202408_Desonerado.xlsx"

dc = "./SINAPI/" + SUFIXO + "_" + estado + "_" + ano + mes + "_" + tipo
arq = dc + "/" + SUFIXO_ARQ + "_" + estado + "_" + ano + mes + "_" + tipo + ".xlsx"

In [56]:
sinapi = pd.read_excel(arq, header=5)

## Trabalhando com o arquivo

In [60]:
sinapi[sinapi["CODIGO DA COMPOSICAO"] == 92265]

,DESCRICAO DA CLASSE,SIGLA DA CLASSE,DESCRICAO DO TIPO 1,SIGLA DO TIPO 1,CODIGO DO AGRUPADOR,DESCRICAO DO AGRUPADOR,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,ORIGEM DE PREÇO,...,% MAO DE OBRA,CUSTO MATERIAL,% MATERIAL,CUSTO EQUIPAMENTO,% EQUIPAMENTO,CUSTO SERVICOS TERCEIROS,% SERVICOS TERCEIROS,CUSTO OUTROS,% OUTROS,VINCULO
14209,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,"24,4798407","75,92","75,4504729","0,02","0,0199104","0,00","0,0000000","0,05","0,0497760",CAIXA REFERENCIAL
14210,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
14211,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
14212,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
14213,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
14214,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
14215,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
14216,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
14217,FUNDACOES E ESTRUTURAS,FUES,FORMAS/CIMBRAMENTOS/ESCORAMENTOS,41.0,NaN,NaN,92265.0,"FABRICAÇÃO DE FÔRMA PARA VIGAS, EM CHAPA DE MA...",M2,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL


In [118]:
# Muda a categoria de código para string ao invés de número
sinapi["CODIGO DA COMPOSICAO"] = sinapi["CODIGO DA COMPOSICAO"].astype(str)
sinapi["CODIGO DA COMPOSICAO"] = sinapi["CODIGO DA COMPOSICAO"].str.replace('.0', '') # remove o .0 que fica após o número
sinapi["CODIGO ITEM"] = sinapi["CODIGO ITEM"].astype(str)
sinapi["CODIGO ITEM"] = sinapi["CODIGO ITEM"].str.replace('.0', '')

In [119]:
for column in sinapi:
    print(column)

DESCRICAO DA CLASSE
SIGLA DA CLASSE
DESCRICAO DO TIPO 1
SIGLA DO TIPO 1
CODIGO DO AGRUPADOR
DESCRICAO DO AGRUPADOR
CODIGO DA COMPOSICAO
DESCRICAO DA COMPOSICAO
UNIDADE
ORIGEM DE PREÇO
CUSTO TOTAL
TIPO ITEM
CODIGO ITEM
DESCRIÇÃO ITEM
UNIDADE ITEM
ORIGEM DE PREÇO ITEM
COEFICIENTE
PRECO UNITARIO
CUSTO TOTAL.1
CUSTO MAO DE OBRA
% MAO DE OBRA
CUSTO MATERIAL
% MATERIAL
CUSTO EQUIPAMENTO
% EQUIPAMENTO
CUSTO SERVICOS TERCEIROS
% SERVICOS TERCEIROS
CUSTO OUTROS
% OUTROS
VINCULO


NOMES DAS COLUNAS - FUNÇÃO
* CODIGO DA COMPOSICAO - código da composição do SINAPI
* DESCRICAO DA COMPOSICAO - descrição da composição do SINAPI
* UNIDADE - unidade da composição
* CUSTO TOTAL - custo total da composição
* TIPO ITEM - tipo do item da composição, para saber se é INSUMO ou COMPOSICAO (se COMPOSICAO abrir até o nível de insumo)
* CODIGO ITEM - código do item da composição
* DESCRIÇÃO ITEM - descrição do item da composição
* UNIDADE ITEM - unidade do item da composição
* COEFICIENTE - coeficiente da composição (ex: 0,235 h/m²)
* PRECO UNITARIO - preço unitário do item
* CUSTO TOTAL.1 - custo total do item (o ".1" é criado porque está com mesmo nome do CUSTO TOTAL da composição)

Não tem coluna que identifica se o insumo é material, mão de obra ou equipamento. Criar uma coluna CLASSE, que identifique isto

In [120]:
COLUMNS = ["CODIGO DA COMPOSICAO",
           "DESCRICAO DA COMPOSICAO",
           "UNIDADE",
           "CUSTO TOTAL",
           "TIPO ITEM",
           "CODIGO ITEM",
           "DESCRIÇÃO ITEM",
           "UNIDADE ITEM",
           "COEFICIENTE",
           "PRECO UNITARIO",
           "CUSTO TOTAL.1",
           "CATEGORIA COMPOSICAO"]

In [121]:
# Convertendo as vírgulas para ponto para o pandas ler como número e removendo o separador de milhar
sinapi["CUSTO TOTAL"] = sinapi["CUSTO TOTAL"].str.replace('.', '')
sinapi["CUSTO TOTAL"] = sinapi["CUSTO TOTAL"].str.replace(',', '.').astype(float)
sinapi["COEFICIENTE"] = sinapi["COEFICIENTE"].str.replace('.', '')
sinapi["COEFICIENTE"] = sinapi["COEFICIENTE"].str.replace(',', '.').astype(float)
sinapi["PRECO UNITARIO"] = sinapi["PRECO UNITARIO"].str.replace('.', '')
sinapi["PRECO UNITARIO"] = sinapi["PRECO UNITARIO"].str.replace(',', '.').astype(float)

In [122]:
# categorização dos itens em equipamento, mão de obra e material para o nível de composição
# QUESTÃO: é necessário??
# PROBLEMA: nem toda unidade H e MÊS refere-se à mão de obra, alguns são locações de equipamento

condicoes = [
    sinapi["UNIDADE"] == "CHP",
    sinapi["UNIDADE"] == "CHI",
    sinapi["UNIDADE"] == "H",
    sinapi["UNIDADE"] == "MÊS",
]

valores = [
    "EQUIPAMENTO",
    "EQUIPAMENTO",
    "MÃO DE OBRA",
    "MÃO DE OBRA",
]

sinapi["CATEGORIA COMPOSICAO"] = np.select(condicoes, valores, default="MATERIAL")

## Simulação

In [123]:
COLUMNS_DF_PLAN = COLUMNS + ["ID", "EAP", "QNTD"]
df_plan = pd.DataFrame(columns = COLUMNS_DF_PLAN)

In [124]:
sinapi[sinapi["CODIGO DA COMPOSICAO"] == "103327"][COLUMNS]

,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,CODIGO ITEM,DESCRIÇÃO ITEM,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO
36974,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,NaN,nan,NaN,NaN,NaN,NaN,NaN,MATERIAL
36975,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.4200,6.05,"2,54",MATERIAL
36976,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.0100,43.74,"0,43",MATERIAL
36977,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37594,BLOCO CERAMICO / TIJOLO VAZADO PARA ALVENARIA ...,UN,13.6000,3.64,"49,50",MATERIAL
36978,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,0.0138,843.36,"11,63",MATERIAL
36979,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,88309,PEDREIRO COM ENCARGOS COMPLEMENTARES,H,0.9900,28.05,"27,76",MÃO DE OBRA
36980,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,88316,SERVENTE COM ENCARGOS COMPLEMENTARES,H,0.4950,21.38,"10,58",MÃO DE OBRA


### 1° elemento

In [125]:
# simulação alvenaria
ID = 1
QS = 200
EAP = 4.1

In [126]:
temp = sinapi[sinapi["CODIGO DA COMPOSICAO"] == "103327"][COLUMNS]
temp["ID"] = ID
temp["EAP"] = EAP
temp["Qs"] = QS

In [127]:
df_plan = pd.concat([df_plan if not df_plan.empty else None, temp], ignore_index=True)

In [128]:
df_plan

,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,CODIGO ITEM,DESCRIÇÃO ITEM,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,ID,EAP,Qs
0,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,NaN,nan,NaN,NaN,NaN,NaN,NaN,MATERIAL,1,4.1,200
1,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.4200,6.05,"2,54",MATERIAL,1,4.1,200
2,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.0100,43.74,"0,43",MATERIAL,1,4.1,200
3,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37594,BLOCO CERAMICO / TIJOLO VAZADO PARA ALVENARIA ...,UN,13.6000,3.64,"49,50",MATERIAL,1,4.1,200
4,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,0.0138,843.36,"11,63",MATERIAL,1,4.1,200
5,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,88309,PEDREIRO COM ENCARGOS COMPLEMENTARES,H,0.9900,28.05,"27,76",MÃO DE OBRA,1,4.1,200
6,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,88316,SERVENTE COM ENCARGOS COMPLEMENTARES,H,0.4950,21.38,"10,58",MÃO DE OBRA,1,4.1,200


### 2° elemento

In [129]:
# simulação alvenaria
ID = 2
QS = 400
EAP = 4.2

In [130]:
temp = sinapi[sinapi["CODIGO DA COMPOSICAO"] == "103327"][COLUMNS]
temp["ID"] = ID
temp["EAP"] = EAP
temp["Qs"] = QS

In [131]:
df_plan = pd.concat([df_plan if not df_plan.empty else None, temp], ignore_index=True)

### Quantidades auxiliares

As quantidades auxiliares são utilizadas pois existem composições auxiliares dentro das composições principais, e esta quantidade que é utilizada nos insumos das composições auxiliares.
Se dentro da composição tiver insumo direto, a quantidade auxiliar corresponde à quantidade total

In [132]:
# Cálculo das quantidades totais das composições auxiliares e insumos
df_plan["QNTD AUX"] = df_plan["COEFICIENTE"] * df_plan["Qs"]
df_plan["QNTD"] = df_plan["COEFICIENTE"] * df_plan["Qs"]
df_plan["RUP"] = ""
df_plan["HISTORICO"] = ""

### Abrindo as composições

In [133]:
df_plan

,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,CODIGO ITEM,DESCRIÇÃO ITEM,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,ID,EAP,Qs,QNTD AUX,QNTD,RUP,HISTORICO
0,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,NaN,nan,NaN,NaN,NaN,NaN,NaN,MATERIAL,1,4.1,200,NaN,NaN,,
1,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.4200,6.05,"2,54",MATERIAL,1,4.1,200,84.00,84.00,,
2,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.0100,43.74,"0,43",MATERIAL,1,4.1,200,2.00,2.00,,
3,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37594,BLOCO CERAMICO / TIJOLO VAZADO PARA ALVENARIA ...,UN,13.6000,3.64,"49,50",MATERIAL,1,4.1,200,2720.00,2720.00,,
4,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,0.0138,843.36,"11,63",MATERIAL,1,4.1,200,2.76,2.76,,
5,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,88309,PEDREIRO COM ENCARGOS COMPLEMENTARES,H,0.9900,28.05,"27,76",MÃO DE OBRA,1,4.1,200,198.00,198.00,,
6,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,COMPOSICAO,88316,SERVENTE COM ENCARGOS COMPLEMENTARES,H,0.4950,21.38,"10,58",MÃO DE OBRA,1,4.1,200,99.00,99.00,,
7,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,NaN,nan,NaN,NaN,NaN,NaN,NaN,MATERIAL,2,4.2,400,NaN,NaN,,
8,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.4200,6.05,"2,54",MATERIAL,2,4.2,400,168.00,168.00,,
9,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.0100,43.74,"0,43",MATERIAL,2,4.2,400,4.00,4.00,,


In [135]:
# Abre as composições
while df_plan[df_plan["TIPO ITEM"] == "COMPOSICAO"].size > 0:
    i = 0
    index_abertos = []
    
    df_item_com_composicao = df_plan[df_plan["TIPO ITEM"] == "COMPOSICAO"]
    composicao_abertas = pd.DataFrame()
    
    while i < df_item_com_composicao.shape[0]:
        temp = sinapi[sinapi["CODIGO DA COMPOSICAO"] == df_item_com_composicao.iloc[i, 5]][COLUMNS]
        temp["ID"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("ID")]
        temp["EAP"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("EAP")]
        temp["Qs"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("Qs")]
        temp["QNTD AUX"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("QNTD")] # QNTD da composição auxiliar vira quantidade do insumo aberto
        temp["HISTORICO"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("CODIGO DA COMPOSICAO")] + "_" + df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("HISTORICO")]
        temp["RUP"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("COEFICIENTE")]
        
        composicao_abertas = pd.concat([composicao_abertas if not composicao_abertas.empty else None, temp], ignore_index=True)
        index_abertos.append(df_item_com_composicao.iloc[i, :].name) # remove a composição que já foi aberta
       
        i += 1
        
    # Exclui as composições abertas
    df_plan.drop(index_abertos, inplace=True)
    
    # Joga as composições abertas no dataframe principal
    df_plan = pd.concat([df_plan if not df_plan.empty else None, composicao_abertas], ignore_index=True)

    # Cálculo das quantidades totais das composições auxiliares e insumos
    df_plan["QNTD"] = df_plan["COEFICIENTE"] * df_plan["QNTD AUX"] # atualiza o valor da QNTD AUX caso tenha mais composições auxiliares (c.a.) dentro da c.a.

### Categorização dos insumos

In [136]:
df_plan.dropna(inplace=True)

In [137]:
# categorização dos itens em equipamento, mão de obra e material

condicoes = [
    df_plan["UNIDADE ITEM"] == "CHP",
    df_plan["UNIDADE ITEM"] == "CHI",
    df_plan["DESCRICAO DA COMPOSICAO"].str.startswith('CURSO'), # O curso de capacitação é categorizado como mão de obra conforme o SINAPI, mas ele não vai ser utilizado como recurso no planejamento
    df_plan["DESCRIÇÃO ITEM"].str.startswith('AJUDANTE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ASSENTADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('AUXILIAR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('AZULEJISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('CALCETEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('CAVOUQUEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('CARPINTEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ELETRICISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ENCANADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ELETROTECNICO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('GESSEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('IMPERMEABILIZADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('INSTALADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('JARDINEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MACARIQUEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MARCENEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MARMORISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MECANICO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MONTADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MOTORISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('NIVELADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('OPERADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('PASTILHEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('PEDREIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('PINTOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('POCEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('SERRALHEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('SERVENTE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('SOLDADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('TOPOGRAFO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('VIDRACEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('VIGIA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ALMOXARIFE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('APONTADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ARQUITETO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('DESENHISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ENCARREGADO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ENGENHEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MESTRE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('TECNICO'), 
]

valores = ["EQUIPAMENTO"]*2 + ["MÃO DE OBRA_CURSO"] + ["MÃO DE OBRA"]*40

df_plan["CATEGORIA INSUMO"] = np.select(condicoes, valores, default="MATERIAL")

In [138]:
df_plan

,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,CODIGO ITEM,DESCRIÇÃO ITEM,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,ID,EAP,Qs,QNTD AUX,QNTD,RUP,HISTORICO,CATEGORIA INSUMO
1,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.42000,6.05,"2,54",MATERIAL,1,4.1,200,84.000,35.280000,,,MATERIAL
2,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.01000,43.74,"0,43",MATERIAL,1,4.1,200,2.000,0.020000,,,MATERIAL
3,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37594,BLOCO CERAMICO / TIJOLO VAZADO PARA ALVENARIA ...,UN,13.60000,3.64,"49,50",MATERIAL,1,4.1,200,2720.000,36992.000000,,,MATERIAL
5,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.42000,6.05,"2,54",MATERIAL,2,4.2,400,168.000,70.560000,,,MATERIAL
6,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.01000,43.74,"0,43",MATERIAL,2,4.2,400,4.000,0.040000,,,MATERIAL
7,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37594,BLOCO CERAMICO / TIJOLO VAZADO PARA ALVENARIA ...,UN,13.60000,3.64,"49,50",MATERIAL,2,4.2,400,5440.000,73984.000000,,,MATERIAL
9,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,843.36,INSUMO,370,AREIA MEDIA - POSTO JAZIDA/FORNECEDOR (RETIRAD...,M3,1.14000,195.00,"222,30",MATERIAL,1,4.1,200,2.760,3.146400,0.0138,103327_,MATERIAL
10,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,843.36,INSUMO,1106,CAL HIDRATADA CH-I PARA ARGAMASSAS,KG,171.13000,1.50,"256,69",MATERIAL,1,4.1,200,2.760,472.318800,0.0138,103327_,MATERIAL
11,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,843.36,INSUMO,1379,CIMENTO PORTLAND COMPOSTO CP II-32,KG,192.52000,0.66,"127,06",MATERIAL,1,4.1,200,2.760,531.355200,0.0138,103327_,MATERIAL
13,88309,PEDREIRO COM ENCARGOS COMPLEMENTARES,H,28.05,INSUMO,4750,PEDREIRO (HORISTA),H,1.00000,18.85,"18,85",MÃO DE OBRA,1,4.1,200,198.000,198.000000,0.99,103327_,MÃO DE OBRA


### Avaliação dos resultados

In [139]:
df_plan

,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,CODIGO ITEM,DESCRIÇÃO ITEM,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,ID,EAP,Qs,QNTD AUX,QNTD,RUP,HISTORICO,CATEGORIA INSUMO
1,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.42000,6.05,"2,54",MATERIAL,1,4.1,200,84.000,35.280000,,,MATERIAL
2,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.01000,43.74,"0,43",MATERIAL,1,4.1,200,2.000,0.020000,,,MATERIAL
3,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37594,BLOCO CERAMICO / TIJOLO VAZADO PARA ALVENARIA ...,UN,13.60000,3.64,"49,50",MATERIAL,1,4.1,200,2720.000,36992.000000,,,MATERIAL
5,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,34548,TELA DE ACO SOLDADA GALVANIZADA/ZINCADA PARA A...,M,0.42000,6.05,"2,54",MATERIAL,2,4.2,400,168.000,70.560000,,,MATERIAL
6,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37395,"PINO DE ACO COM FURO, HASTE = 27 MM (ACAO DIRETA)",CENTO,0.01000,43.74,"0,43",MATERIAL,2,4.2,400,4.000,0.040000,,,MATERIAL
7,103327,ALVENARIA DE VEDAÇÃO DE BLOCOS CERÂMICOS FURAD...,M2,102.44,INSUMO,37594,BLOCO CERAMICO / TIJOLO VAZADO PARA ALVENARIA ...,UN,13.60000,3.64,"49,50",MATERIAL,2,4.2,400,5440.000,73984.000000,,,MATERIAL
9,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,843.36,INSUMO,370,AREIA MEDIA - POSTO JAZIDA/FORNECEDOR (RETIRAD...,M3,1.14000,195.00,"222,30",MATERIAL,1,4.1,200,2.760,3.146400,0.0138,103327_,MATERIAL
10,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,843.36,INSUMO,1106,CAL HIDRATADA CH-I PARA ARGAMASSAS,KG,171.13000,1.50,"256,69",MATERIAL,1,4.1,200,2.760,472.318800,0.0138,103327_,MATERIAL
11,87369,"ARGAMASSA TRAÇO 1:2:8 (EM VOLUME DE CIMENTO, C...",M3,843.36,INSUMO,1379,CIMENTO PORTLAND COMPOSTO CP II-32,KG,192.52000,0.66,"127,06",MATERIAL,1,4.1,200,2.760,531.355200,0.0138,103327_,MATERIAL
13,88309,PEDREIRO COM ENCARGOS COMPLEMENTARES,H,28.05,INSUMO,4750,PEDREIRO (HORISTA),H,1.00000,18.85,"18,85",MÃO DE OBRA,1,4.1,200,198.000,198.000000,0.99,103327_,MÃO DE OBRA


In [140]:
df_plan[df_plan["CATEGORIA INSUMO"] == "MÃO DE OBRA"]

,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,CODIGO ITEM,DESCRIÇÃO ITEM,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,ID,EAP,Qs,QNTD AUX,QNTD,RUP,HISTORICO,CATEGORIA INSUMO
13,88309,PEDREIRO COM ENCARGOS COMPLEMENTARES,H,28.05,INSUMO,4750,PEDREIRO (HORISTA),H,1.0,18.85,"18,85",MÃO DE OBRA,1,4.1,200,198.000,198.000,0.99,103327_,MÃO DE OBRA
21,88316,SERVENTE COM ENCARGOS COMPLEMENTARES,H,21.38,INSUMO,6111,SERVENTE DE OBRAS (HORISTA),H,1.0,12.46,"12,46",MÃO DE OBRA,1,4.1,200,99.000,99.000,0.495,103327_,MÃO DE OBRA
33,88309,PEDREIRO COM ENCARGOS COMPLEMENTARES,H,28.05,INSUMO,4750,PEDREIRO (HORISTA),H,1.0,18.85,"18,85",MÃO DE OBRA,2,4.2,400,396.000,396.000,0.99,103327_,MÃO DE OBRA
41,88316,SERVENTE COM ENCARGOS COMPLEMENTARES,H,21.38,INSUMO,6111,SERVENTE DE OBRAS (HORISTA),H,1.0,12.46,"12,46",MÃO DE OBRA,2,4.2,400,198.000,198.000,0.495,103327_,MÃO DE OBRA
49,88316,SERVENTE COM ENCARGOS COMPLEMENTARES,H,21.38,INSUMO,6111,SERVENTE DE OBRAS (HORISTA),H,1.0,12.46,"12,46",MÃO DE OBRA,1,4.1,200,30.636,30.636,11.1,87369_103327_,MÃO DE OBRA
61,88316,SERVENTE COM ENCARGOS COMPLEMENTARES,H,21.38,INSUMO,6111,SERVENTE DE OBRAS (HORISTA),H,1.0,12.46,"12,46",MÃO DE OBRA,2,4.2,400,61.272,61.272,11.1,87369_103327_,MÃO DE OBRA


# Associar SINAPI ao IFC

## IFC-Revit-TQS

Não se tem projeto de forma, de armação e de escoramento, estas quantidades vão ser estimadas pelo projeto de estrutura somente

### Criação da EAP

In [12]:
# separadores
SEPARADOR_DESCRICAO = " - "
SEPARADOR_EAP = "."

### Etiquetagem dos elementos com a EAP concretagem

In [14]:
df_concreto = df_quantitativo.copy()

In [15]:
# criação de colunas para identificar os itens e padronizar a organização
df_concreto["ID"] = ""
df_concreto["EAP"] = ""
df_concreto["descrição"] = ""
df_concreto["local"] = "Edifício"
df_concreto["serviço"] = "Concretagem"

# agrupa os elementos em pilares e mãos francesas, vigas e lajes, e cortina
df_concreto["elemento_agrupado"] = df_concreto["elemento"]
df_concreto.loc[(df_concreto["elemento"] == "Laje") | (df_concreto["elemento"] == "Capitel") | (df_concreto["elemento"] == "Viga"), "elemento_agrupado"] = "Lajes e Vigas"

In [16]:
# a divisão da EAP será dada por juntas - elementos - pavimentos

locais = np.array(["Edifício"]) #### PENDENCIA criar este parâmetro de local no IFC como fiz com as juntas
serviços = np.array(["Concretagem"])

juntas = np.sort(df_concreto["junta"].unique())
elementos = np.sort(df_concreto["elemento_agrupado"].unique())
pavimentos = np.sort(df_concreto["pavimento"].unique())

# Cria um DataFrame auxiliar com todas as combinações que serão dados o número de EAP e descrição
combinacoes = pd.MultiIndex.from_product(
    [locais, juntas, elementos, serviços, pavimentos],
    names=["local", "junta", "elemento_agrupado", "serviço", "pavimento"]
).to_frame(index=False)

# Cria a coluna EAP
combinacoes["EAP"] = (
    combinacoes["local"].apply(lambda x: str(np.where(locais == x)[0][0] + 1)) + SEPARADOR_EAP +
    combinacoes["junta"].apply(lambda x: str(np.where(juntas == x)[0][0] + 1)) + SEPARADOR_EAP +
    combinacoes["elemento_agrupado"].apply(lambda x: str(np.where(elementos == x)[0][0] + 1)) + SEPARADOR_EAP +
    combinacoes["serviço"].apply(lambda x: str(np.where(serviços == x)[0][0] + 1)) + SEPARADOR_EAP +
    combinacoes["pavimento"].apply(lambda x: str(np.where(pavimentos == x)[0][0] + 1))
)

# Gera as descrições e EAPs para cada combinação
combinacoes["descrição"] = (
    combinacoes["local"] + SEPARADOR_DESCRICAO +
    combinacoes["junta"] + SEPARADOR_DESCRICAO +
    combinacoes["elemento_agrupado"] + SEPARADOR_DESCRICAO +
    combinacoes["serviço"] + SEPARADOR_DESCRICAO +
    combinacoes["pavimento"]
)


# Atualiza o DataFrame
for idx, row in combinacoes.iterrows():
    df_concreto.loc[
        (df_concreto["local"] == row["local"]) &
        (df_concreto["junta"] == row["junta"]) &
        (df_concreto["elemento_agrupado"] == row["elemento_agrupado"]) &
        (df_concreto["serviço"] == row["serviço"]) &
        (df_concreto["pavimento"] == row["pavimento"]),
        ["descrição", "EAP"]
    ] = row["descrição"], row["EAP"]

In [17]:
# EAP em cada um dos elementos para ser inserido no IFC
eap_plan = df_concreto[["GlobalID", "EAP", "descrição"]]

In [18]:
eap_plan.to_excel("EST-EAP.xlsx")

### Associação ao SINAPI

### Criação do cronograma

In [47]:
# a divisão da EAP será dada por juntas - elementos - pavimentos

locais = np.array(["Edifício"]) #### PENDENCIA criar este parâmetro de local no IFC como fiz com as juntas
serviços = np.array(["Concretagem"])

juntas = np.sort(df_concreto["junta"].unique())
elementos = np.sort(df_concreto["elemento_agrupado"].unique())
pavimentos = np.sort(df_concreto["pavimento"].unique())

# Nível 2 EAP
combinacoes_2 = criar_combinacoes(["local", "junta"], locais, juntas)

# Nível 3 EAP
combinacoes_3 = criar_combinacoes(["local", "junta", "elemento_agrupado"], locais, juntas, elementos)

# Nível 4 EAP
combinacoes_4 = criar_combinacoes(["local", "junta", "elemento_agrupado", "serviço"], locais, juntas, elementos, serviços)

# Nível 5 EAP
### Especifico para a POE 789, pois tem juntas com quantidade diferentes de pavimento
### MELHORIA: verificar como automatizar a verificação para não ser feita a separação de juntas com quantidade diferentes de pavimento manualmente
combinacoes_5a = criar_combinacoes(["local", "junta", "elemento_agrupado", "serviço", "pavimento"], locais, juntas[:-2], elementos, serviços, pavimentos) # exclui as juntas E e F
combinacoes_5b = criar_combinacoes(["local", "junta", "elemento_agrupado", "serviço", "pavimento"],
                                   locais, juntas[-2:], elementos, serviços, np.array([pavimentos[0]])) # inclui as juntas E e F

# incrementar o nível da EAP na posição 1 de junta somando 4
### Especifico para a POE 789
combinacoes_5b["EAP"] = combinacoes_5b["EAP"].apply(lambda x: incrementar_eap(x, 1, 4))

# Concatena todos os DataFrames de EAP
df_eap = pd.concat([combinacoes_2[["EAP", "descrição"]],
                 combinacoes_3[["EAP", "descrição"]],
                 combinacoes_4[["EAP", "descrição"]],
                 combinacoes_5a[["EAP", "descrição"]],
                 combinacoes_5b[["EAP", "descrição"]]])

# Ordena o resultado final com base na numeração da EAP
df_eap.sort_values(by=["EAP"], inplace=True, ignore_index=True)

In [48]:
df_eap

,EAP,descrição
0,1.1,Edifício - Junta A
1,1.1.1,Edifício - Junta A - Cortina
2,1.1.1.1,Edifício - Junta A - Cortina - Concretagem
3,1.1.1.1.1,Edifício - Junta A - Cortina - Concretagem - 0...
4,1.1.1.1.2,Edifício - Junta A - Cortina - Concretagem - 0...
...,...,...
151,1.6.2.1,Edifício - Junta F - Lajes e Vigas - Concretagem
152,1.6.2.1.1,Edifício - Junta F - Lajes e Vigas - Concretag...
153,1.6.3,Edifício - Junta F - Pilar
154,1.6.3.1,Edifício - Junta F - Pilar - Concretagem


In [49]:
# exporta a eap
df_eap.to_excel("teste.xlsx")

In [50]:
# criação do plano de trabalho e cronograma
construction_work_plan = ifcopenshell.api.sequence.add_work_plan(model, name="Construction", predefined_type="PLANNED")
construction_work_schedule = ifcopenshell.api.sequence.add_work_schedule(model, name="POE 789_Construção_Planejado",
                                                                    work_plan=construction_work_plan, predefined_type="PLANNED")

# criação da tarefa resumo
summary_task = ifcopenshell.api.sequence.add_task(model, work_schedule=construction_work_schedule,
                                       name = "POE 789 - Construção",
                                       identification = "1")

In [51]:
# Criação do calendário
calendar_default = ifcopenshell.api.sequence.add_work_calendar(model, name="5 dias")

# Horários de trabalho na semana
work_time = ifcopenshell.api.sequence.add_work_time(model, work_calendar=calendar_default, time_type="WorkingTimes")

# Criação da recorrência semanal de trabalho
pattern = ifcopenshell.api.sequence.assign_recurrence_pattern(model, parent=work_time, recurrence_type="WEEKLY")

# Estabelece os horários e dias de trabalho
# Segunda a quinta de 07:00 às 17:00 e sexta 07:00 às 16:00
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"WeekdayComponent": [1, 2, 3, 4]})
ifcopenshell.api.sequence.add_time_period(model, recurrence_pattern=pattern, start_time="07:00", end_time="17:00")
#ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"WeekdayComponent": [5]})
#ifcopenshell.api.sequence.add_time_period(model, recurrence_pattern=pattern, start_time="07:00", end_time="16:00")

#167503=IfcTimePeriod('07:00:00','17:00:00')

In [52]:
# Feriados
holidays = ifcopenshell.api.sequence.add_work_time(model, work_calendar=calendar_default, time_type="ExceptionTimes")

# Criação da recorrência anual por dia do mês
pattern = ifcopenshell.api.sequence.assign_recurrence_pattern(model, parent=work_time, recurrence_type="YEARLY_BY_DAY_OF_MONTH")

# Feriado dia 1° de Janeiro - 01/01
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [1], "MonthComponent": [1]})

# Feriado Dia de Tiradentes / Aniversário de Brasília - 21/04
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [21], "MonthComponent": [4]})

# Feriado Dia Mundial do Trabalho (feriado nacional) - 01/05
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [1], "MonthComponent": [5]})

# Dia da Independência do Brasil (feriado nacional) - 07/09
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [7], "MonthComponent": [9]})

# Dia de N. Sª. Aparecida (padroeira do Brasil) - 12/10
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [12], "MonthComponent": [10]})

# Dia de Finados (feriado nacional) - 02/11
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [2], "MonthComponent": [11]})

# Proclamação da República (feriado nacional) - 15/11
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [15], "MonthComponent": [11]})

# Dia Nacional de Zumbi e da Consciência Negra (feriado nacional) - 20/11
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [20], "MonthComponent": [11]})

# Dia do Evangélico (feriado local) - 30/11
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [30], "MonthComponent": [11]})

# Natal (feriado nacional) - 25/12
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"DayComponent": [25], "MonthComponent": [12]})


# Criação da recorrência anual por posição
pattern = ifcopenshell.api.sequence.assign_recurrence_pattern(model, parent=work_time, recurrence_type="YEARLY_BY_POSITION")

# Feriado Dia de São José – Padroeiro da Construção Civil – comemorado na segunda-feira de Carnaval (data móvel) - 12/02 Segunda-feira
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 1, "WeekdayComponent": [2], "MonthComponent": [2]})

# Feriado Paixão de Cristo (data móvel) - 29/03/2024 Sexta-feira
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 5, "WeekdayComponent": [4], "MonthComponent": [3]})

# Feriado Corpus Christi (data móvel) - 30/05/2024 Quinta-feira
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 4, "WeekdayComponent": [4], "MonthComponent": [5]})

# Férias coletivas PO (data móvel) - quarta 18/12/2024 a domingo 05/01/2025
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 3, "WeekdayComponent": [3], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 4, "WeekdayComponent": [3], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 5, "WeekdayComponent": [3], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 6, "WeekdayComponent": [3], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 7, "WeekdayComponent": [3], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 1, "WeekdayComponent": [4], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 2, "WeekdayComponent": [4], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 3, "WeekdayComponent": [4], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 4, "WeekdayComponent": [4], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 5, "WeekdayComponent": [4], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 6, "WeekdayComponent": [4], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 7, "WeekdayComponent": [4], "MonthComponent": [12]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 1, "WeekdayComponent": [1], "MonthComponent": [1]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 2, "WeekdayComponent": [1], "MonthComponent": [1]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 3, "WeekdayComponent": [1], "MonthComponent": [1]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 4, "WeekdayComponent": [1], "MonthComponent": [1]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 5, "WeekdayComponent": [1], "MonthComponent": [1]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 6, "WeekdayComponent": [1], "MonthComponent": [1]})
ifcopenshell.api.sequence.edit_recurrence_pattern(model, recurrence_pattern=pattern, attributes={"Position": 7, "WeekdayComponent": [1], "MonthComponent": [1]})

In [53]:
# Adiciona o calendário na tarefa resumo e este calendário é herdado para todas as tarefas
ifcopenshell.api.control.assign_control(model, relating_control=calendar_default, related_object=summary_task)

#167508=IfcRelAssignsToControl('0KJn_i_sj9rxW7IXly0Gqz',#167507,$,$,(#167496),$,#167500)

In [54]:
# criação das tarefas e subtarefas associadas entre si de acordo com a eap
for eap in df_eap["EAP"]:
    eap_superior = '.'.join(eap.split('.')[:-1])
    parent_task = ifcopenshell.util.selector.filter_elements(model, 'IfcTask, Identification="{}"'.format(eap_superior)).pop()
    ifcopenshell.api.sequence.add_task(model,
                                       name = df_eap.loc[df_eap["EAP"] == eap, "descrição"].to_numpy()[0],
                                       identification = eap,
                                       parent_task = parent_task,
                                       predefined_type="CONSTRUCTION"
                                      )


In [55]:
# escreve o modelo IFC em um novo arquivo
model.write("./modelo_teste/est_eap.ifc")

## QiBuilder

In [213]:
df_quantitativo

,pavimento,rede,item,quantidade,ID
0,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R
1,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,23vghV9i5E69JyH8cCXCzC
2,TER NA,Alimentação (Piscina-FIL),"Metais - Valvula de retenção vertical - 1.1/2""",1.0,3Xe7iW2cXEUfq5MHSwOCQP
3,TER NA,Alimentação (Piscina-FIL),PVC rígido soldável - Adapt sold.curto c/bolsa...,2.0,3Xe7iW2cXEUfq5MHSwOCQP
4,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,3mVMY5yAD9uAhX6Sme9ohP
...,...,...,...,...,...
552,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,1K8ErAvxDEhuVB3L0$rilu
553,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,2gUdsZucfD09_ho9YhCYi8
554,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,1HEBKtuuD6PhkNQxhOCT7h
555,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,2c8JUCCDjFr8EWB5NyNC1Q


In [214]:
######## PENDENCIA
######## Categorizar os itens entre prumada, ramal, subcoletor, reservação, etc.
# itens únicos dentro do dataframe
df_quantitativo["item"].sort_values().unique()

array(['Acessórios para Piscina - Dispositivo de Aspiração - Disp. de Aspiração p/ Pisc. de Concreto e Fibra - Qmáx = 9m³/h',
       'Acessórios para Piscina - Dispositivo de Retorno - Disp. Retorno p/ Pisc. de Concreto ou Fibra - Qmín = 1,4 m³/h e Qmáx = 2,3 m³/h',
       'Acessórios para Piscina - Dispositivo de Retorno - Disp. Retorno p/ Pisc. de Concreto ou Fibra - Qmín = 2,2m³/h e Qmáx = 3,8 m³/h',
       'Acessórios para Piscina - Dreno de Fundo - Dreno c/ Tampa FSB Piscina de Concreto ou Fibra - Qmáx = 30m³/h',
       'Acessórios para Piscina - Skimmer - Skimmer BL p/ Piscina de Concreto ou Vinil',
       "Bomba Hidráulica - Piscina - Autoescorvante com Pré Filtro 1 1/2'' x 1 1/2'' - PF-17 0.33CV R96",
       "Bomba Hidráulica - Piscina - Autoescorvante com Pré Filtro 1 1/2'' x 1 1/2'' - PF-17 0.5CV R100",
       'Caixas de Passagem - Caixa de areia pluvial - CA 60x60x50cm',
       'Filtro de piscina - Série TP - 15-TP',
       'Metais - Filtro em Y - 1.1/2"',
       'Metais - R

In [215]:
mapeamento= {
    'Bomba Hidráulica - Piscina - Autoescorvante com Pré Filtro 1 1/2'' x 1 1/2'' - PF-17 0.33CV R96': "102111",
    'Bomba Hidráulica - Piscina - Autoescorvante com Pré Filtro 1 1/2'' x 1 1/2'' - PF-17 0.5CV R100': "102111",
    'Caixas de Passagem - Caixa de areia pluvial - CA 60x60x50cm': "99260",
    'Metais - Registro esfera VS compacto soldável PVC - 50 mm': "94492",
    'Metais - Registro esfera VS compacto soldável PVC - 60 mm': "94493",
    'Metais - Valvula de retenção vertical - 1.1/2"': "99631",
    'Metais - Valvula de retenção vertical - 2"': "99632",
    'PVC Esgoto - Anel de borracha - 100mm - 4"': "Insumo de composição",
    'PVC Esgoto - Anel de borracha - 50mm - 2"': "Insumo de composição",
    'Curva 90 longa - 50 mm': "89735",
    'PVC Esgoto - Joelho 90 - 100 mm': "89744",
    'PVC Esgoto - Luva simples - 100 mm': "89778",
    'PVC Esgoto - Luva simples - 50 mm': "89753",
    'PVC Esgoto - Terminal de ventilação - 50 mm': "104348",
    'PVC Esgoto - Tubo rígido c/ ponta lisa - 100 mm - 4"': "89714",
    'PVC Esgoto - Tubo rígido c/ ponta lisa - 50 mm - 2"': "89712",
    'PVC rígido soldável - Adapt sold.curto c/bolsa-rosca p registro - 50 mm - 1.1/2"': "94662",
    'PVC rígido soldável - Adapt sold.curto c/bolsa-rosca p registro - 60 mm - 2"': "94664",
    'PVC rígido soldável - Bucha de redução sold. curta - 60 mm - 50 mm': "103959",
    'PVC rígido soldável - Bucha de redução sold. longa - 50 mm - 25 mm': "105234",
    'PVC rígido soldável - Curva 45 soldável - 50 mm': "103987",
    'PVC rígido soldável - Curva 45 soldável - 60 mm': "89510",
    'PVC rígido soldável - Curva 90 soldável - 50 mm': "94679",
    'PVC rígido soldável - Curva 90 soldável - 60 mm': "94681",
    'PVC rígido soldável - Joelho 45 soldável - 60 mm': "89506",
    'PVC rígido soldável - Joelho 90º soldável - 50 mm': "94678",
    'PVC rígido soldável - Joelho 90º soldável - 60 mm': "94680",
    'PVC rígido soldável - Luva soldável - 50 mm': "94663",
    'PVC rígido soldável - Luva soldável - 60 mm': "94665",
    'PVC rígido soldável - Tubos - 25 mm': "89402",
    'PVC rígido soldável - Tubos - 50 mm': "94651",
    'PVC rígido soldável - Tubos - 60 mm': "94652",
    'PVC rígido soldável - Tê 90 soldável - 50 mm': "94694",
    'PVC rígido soldável - Tê 90 soldável - 60 mm': "94696",
    'PVC rígido soldável - União soldável - 50 mm': "103997",
    'PVC rígido soldável - União soldável - 60 mm': "89609"  
}

df_quantitativo["CODIGO DA COMPOSICAO"] = df_quantitativo["item"].map(mapeamento).fillna("não associado")

In [216]:
df_quantitativo

,pavimento,rede,item,quantidade,ID,CODIGO DA COMPOSICAO
0,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492
1,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,23vghV9i5E69JyH8cCXCzC,94492
2,TER NA,Alimentação (Piscina-FIL),"Metais - Valvula de retenção vertical - 1.1/2""",1.0,3Xe7iW2cXEUfq5MHSwOCQP,99631
3,TER NA,Alimentação (Piscina-FIL),PVC rígido soldável - Adapt sold.curto c/bolsa...,2.0,3Xe7iW2cXEUfq5MHSwOCQP,94662
4,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,3mVMY5yAD9uAhX6Sme9ohP,94492
...,...,...,...,...,...,...
552,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,1K8ErAvxDEhuVB3L0$rilu,não associado
553,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,2gUdsZucfD09_ho9YhCYi8,não associado
554,COBERTURA,Água quente,Placa Solar - Solis - New Trópicos 2000,1.0,1HEBKtuuD6PhkNQxhOCT7h,não associado
555,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,2c8JUCCDjFr8EWB5NyNC1Q,104348


In [217]:
# Associar os quantidades com os códigos do sinapi
df_plan = df_quantitativo.merge(sinapi[COLUMNS], on="CODIGO DA COMPOSICAO", how="left")

In [218]:
df_plan

,pavimento,rede,item,quantidade,ID,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,CODIGO ITEM,DESCRIÇÃO ITEM,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO
0,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,NaN,nan,NaN,NaN,NaN,NaN,NaN,MATERIAL
1,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,11677,"REGISTRO DE ESFERA, PVC, COM VOLANTE, VS, SOLD...",UN,1.0000,66.05,"66,05",MATERIAL
2,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,20080,"ADESIVO PLASTICO PARA PVC, FRASCO COM 175 GR",UN,0.0714,21.37,"1,52",MATERIAL
3,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,20083,"SOLUCAO PREPARADORA / LIMPADORA PARA PVC, FRAS...",UN,0.0180,74.18,"1,33",MATERIAL
4,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,38383,"LIXA D'AGUA EM FOLHA, GRAO 100",UN,0.0114,2.51,"0,02",MATERIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,INSUMO,20083,"SOLUCAO PREPARADORA / LIMPADORA PARA PVC, FRAS...",UN,0.0110,74.18,"0,81",MATERIAL
3006,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,INSUMO,38383,"LIXA D'AGUA EM FOLHA, GRAO 100",UN,0.0080,2.51,"0,02",MATERIAL
3007,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,INSUMO,39319,"TERMINAL DE VENTILACAO, 50 MM, SERIE NORMAL, E...",UN,1.0000,8.36,"8,36",MATERIAL
3008,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,COMPOSICAO,88248,AUXILIAR DE ENCANADOR OU BOMBEIRO HIDRÁULICO C...,H,0.0114,21.51,"0,24",MÃO DE OBRA


# APAGAR SE OS CÓDIGOS ACIMA DEREM CERTO

### Quantidades auxiliares

As quantidades auxiliares são utilizadas pois existem composições auxiliares dentro das composições principais, e esta quantidade que é utilizada nos insumos das composições auxiliares.
Se dentro da composição tiver insumo direto, a quantidade auxiliar corresponde à quantidade total

In [219]:
# Cálculo das quantidades totais das composições auxiliares e insumos
df_plan["QNTD AUX"] = df_plan["COEFICIENTE"] * df_plan["quantidade"]
df_plan["QNTD"] = df_plan["COEFICIENTE"] * df_plan["quantidade"]
df_plan["EAP"] = ""
df_plan["RUP"] = ""
df_plan["HISTORICO"] = ""

### Abrindo as composições

In [220]:
df_plan

,pavimento,rede,item,quantidade,ID,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,...,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,QNTD AUX,QNTD,EAP,RUP,HISTORICO
0,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,NaN,...,NaN,NaN,NaN,NaN,MATERIAL,NaN,NaN,,,
1,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,1.0000,66.05,"66,05",MATERIAL,1.0000,1.0000,,,
2,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,0.0714,21.37,"1,52",MATERIAL,0.0714,0.0714,,,
3,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,0.0180,74.18,"1,33",MATERIAL,0.0180,0.0180,,,
4,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,0.0114,2.51,"0,02",MATERIAL,0.0114,0.0114,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,INSUMO,...,UN,0.0110,74.18,"0,81",MATERIAL,0.0110,0.0110,,,
3006,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,INSUMO,...,UN,0.0080,2.51,"0,02",MATERIAL,0.0080,0.0080,,,
3007,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,INSUMO,...,UN,1.0000,8.36,"8,36",MATERIAL,1.0000,1.0000,,,
3008,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.0,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,COMPOSICAO,...,H,0.0114,21.51,"0,24",MÃO DE OBRA,0.0114,0.0114,,,


In [221]:
df_item_com_composicao = df_plan[df_plan["TIPO ITEM"] == "COMPOSICAO"]

In [222]:
df_item_com_composicao

,pavimento,rede,item,quantidade,ID,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,...,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,QNTD AUX,QNTD,EAP,RUP,HISTORICO
5,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.000000,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,COMPOSICAO,...,H,0.1133,21.51,"2,43",MÃO DE OBRA,0.113300,0.113300,,,
6,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.000000,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,COMPOSICAO,...,H,0.1133,27.29,"3,09",MÃO DE OBRA,0.113300,0.113300,,,
12,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.000000,23vghV9i5E69JyH8cCXCzC,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,COMPOSICAO,...,H,0.1133,21.51,"2,43",MÃO DE OBRA,0.113300,0.113300,,,
13,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.000000,23vghV9i5E69JyH8cCXCzC,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,COMPOSICAO,...,H,0.1133,27.29,"3,09",MÃO DE OBRA,0.113300,0.113300,,,
17,TER NA,Alimentação (Piscina-FIL),"Metais - Valvula de retenção vertical - 1.1/2""",1.000000,3Xe7iW2cXEUfq5MHSwOCQP,99631,"VÁLVULA DE RETENÇÃO VERTICAL, DE BRONZE, ROSCÁ...",UN,154.72,COMPOSICAO,...,H,0.2633,21.51,"5,66",MÃO DE OBRA,0.263300,0.263300,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979,COBERTURA,Alimentação (Piscina-AQ),PVC rígido soldável - Tubos - 60 mm,110.457159,0$zGCv7zb4nAlDpBS3I3Np,94652,"TUBO, PVC, SOLDÁVEL, DE 60MM, INSTALADO EM RES...",M,32.77,COMPOSICAO,...,H,0.1517,27.29,"4,13",MÃO DE OBRA,16.756351,16.756351,,,
3001,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.000000,2c8JUCCDjFr8EWB5NyNC1Q,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,COMPOSICAO,...,H,0.0114,21.51,"0,24",MÃO DE OBRA,0.011400,0.011400,,,
3002,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.000000,2c8JUCCDjFr8EWB5NyNC1Q,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,COMPOSICAO,...,H,0.0114,27.29,"0,31",MÃO DE OBRA,0.011400,0.011400,,,
3008,TER NA,Ventilação,PVC Esgoto - Terminal de ventilação - 50 mm,1.000000,3hPDKMePT1yAVb4qJWCNuJ,104348,"TERMINAL DE VENTILAÇÃO, PVC, SÉRIE NORMAL, ESG...",UN,10.21,COMPOSICAO,...,H,0.0114,21.51,"0,24",MÃO DE OBRA,0.011400,0.011400,,,


# Abre as composições
while df_plan[df_plan["TIPO ITEM"] == "COMPOSICAO"].size > 0:
    i = 0
    index_abertos = []
    
    df_item_com_composicao = df_plan[df_plan["TIPO ITEM"] == "COMPOSICAO"]
    composicao_abertas = pd.DataFrame()
    
    while i < df_item_com_composicao.shape[0]:
        temp = sinapi[sinapi["CODIGO DA COMPOSICAO"] == df_item_com_composicao.iloc[i, 5]][COLUMNS]
        temp["ID"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("ID")]
        temp["EAP"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("EAP")]
        temp["pavimento"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("pavimento")]
        temp["rede"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("rede")]
        temp["quantidade"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("quantidade")]
        temp["QNTD AUX"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("QNTD")] # QNTD da composição auxiliar vira quantidade do insumo aberto
        temp["HISTORICO"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("CODIGO DA COMPOSICAO")] + "_" + df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("HISTORICO")]
        temp["RUP"] = df_item_com_composicao.iloc[i, df_item_com_composicao.columns.get_loc("COEFICIENTE")]
        
        composicao_abertas = pd.concat([composicao_abertas if not composicao_abertas.empty else None, temp], ignore_index=True)
        index_abertos.append(df_item_com_composicao.iloc[i, :].name) # remove a composição que já foi aberta
       
        i += 1
        
    # Exclui as composições abertas
    df_plan.drop(index_abertos, inplace=True)
    
    # Joga as composições abertas no dataframe principal
    df_plan = pd.concat([df_plan if not df_plan.empty else None, composicao_abertas], ignore_index=True)

    # Cálculo das quantidades totais das composições auxiliares e insumos
    df_plan["QNTD"] = df_plan["COEFICIENTE"] * df_plan["QNTD AUX"] # atualiza o valor da QNTD AUX caso tenha mais composições auxiliares (c.a.) dentro da c.a.

In [224]:
# Abre as composições
while not df_plan[df_plan["TIPO ITEM"] == "COMPOSICAO"].empty:
    df_item_com_composicao = df_plan[df_plan["TIPO ITEM"] == "COMPOSICAO"]
    
    # Extraindo as colunas relevantes apenas uma vez
    codigos_composicao = df_item_com_composicao["CODIGO DA COMPOSICAO"].values
    ids = df_item_com_composicao["ID"].values
    eaps = df_item_com_composicao["EAP"].values
    pavimentos = df_item_com_composicao["pavimento"].values
    redes = df_item_com_composicao["rede"].values
    quantidades = df_item_com_composicao["quantidade"].values
    qntd_auxs = df_item_com_composicao["QNTD"].values
    coeficientes = df_item_com_composicao["COEFICIENTE"].values
    
    # Buscando dados no sinapi uma única vez
    temp_data = sinapi[sinapi["CODIGO DA COMPOSICAO"].isin(codigos_composicao)][COLUMNS]

    # Criando uma nova DataFrame para as composições abertas
    composicao_abertas = pd.DataFrame({
        "ID": ids,
        "EAP": eaps,
        "pavimento": pavimentos,
        "rede": redes,
        "quantidade": quantidades,
        "QNTD AUX": qntd_auxs,
        "HISTORICO": [f"{cod}_{hist}" for cod, hist in zip(codigos_composicao, df_item_com_composicao["HISTORICO"].values)],
        "RUP": coeficientes
    })

    # Atualiza df_plan excluindo as composições abertas
    df_plan = df_plan[~df_plan.index.isin(df_item_com_composicao.index)]

    # Joga as composições abertas no dataframe principal
    df_plan = pd.concat([df_plan, composicao_abertas], ignore_index=True)

    # Cálculo das quantidades totais das composições auxiliares e insumos
    df_plan["QNTD"] = df_plan["COEFICIENTE"] * df_plan["QNTD AUX"]

In [225]:
df_plan

,pavimento,rede,item,quantidade,ID,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,CUSTO TOTAL,TIPO ITEM,...,UNIDADE ITEM,COEFICIENTE,PRECO UNITARIO,CUSTO TOTAL.1,CATEGORIA COMPOSICAO,QNTD AUX,QNTD,EAP,RUP,HISTORICO
0,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,NaN,...,NaN,NaN,NaN,NaN,MATERIAL,NaN,NaN,,,
1,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,1.0000,66.05,"66,05",MATERIAL,1.000000e+00,1.000000,,,
2,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,0.0714,21.37,"1,52",MATERIAL,7.140000e-02,0.005098,,,
3,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,0.0180,74.18,"1,33",MATERIAL,1.800000e-02,0.000324,,,
4,TER NA,Alimentação (Piscina-FIL),Metais - Registro esfera VS compacto soldável ...,1.0,1yH6iSt2HEAwhBHmpEW48R,94492,"REGISTRO DE ESFERA, PVC, SOLDÁVEL, COM VOLANTE...",UN,74.44,INSUMO,...,UN,0.0114,2.51,"0,02",MATERIAL,1.140000e-02,0.000130,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150596,TER NA,Ventilação,NaN,1.0,3hPDKMePT1yAVb4qJWCNuJ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.194973e-12,NaN,,0.0114,104348_104348_104348_104348_104348_104348_
1150597,TER NA,Ventilação,NaN,1.0,3hPDKMePT1yAVb4qJWCNuJ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.194973e-12,NaN,,0.0114,104348_104348_104348_104348_104348_104348_
1150598,TER NA,Ventilação,NaN,1.0,3hPDKMePT1yAVb4qJWCNuJ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.194973e-12,NaN,,0.0114,104348_104348_104348_104348_104348_104348_
1150599,TER NA,Ventilação,NaN,1.0,3hPDKMePT1yAVb4qJWCNuJ,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.194973e-12,NaN,,0.0114,104348_104348_104348_104348_104348_104348_


### Categorização dos insumos

In [ ]:
df_plan.dropna(inplace=True)

In [ ]:
# categorização dos itens em equipamento, mão de obra e material

condicoes = [
    df_plan["UNIDADE ITEM"] == "CHP",
    df_plan["UNIDADE ITEM"] == "CHI",
    df_plan["DESCRICAO DA COMPOSICAO"].str.startswith('CURSO'), # O curso de capacitação é categorizado como mão de obra conforme o SINAPI, mas ele não vai ser utilizado como recurso no planejamento
    df_plan["DESCRIÇÃO ITEM"].str.startswith('AJUDANTE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ASSENTADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('AUXILIAR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('AZULEJISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('CALCETEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('CAVOUQUEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('CARPINTEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ELETRICISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ENCANADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ELETROTECNICO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('GESSEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('IMPERMEABILIZADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('INSTALADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('JARDINEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MACARIQUEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MARCENEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MARMORISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MECANICO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MONTADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MOTORISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('NIVELADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('OPERADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('PASTILHEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('PEDREIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('PINTOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('POCEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('SERRALHEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('SERVENTE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('SOLDADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('TOPOGRAFO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('VIDRACEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('VIGIA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ALMOXARIFE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('APONTADOR'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ARQUITETO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('DESENHISTA'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ENCARREGADO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('ENGENHEIRO'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('MESTRE'),
    df_plan["DESCRIÇÃO ITEM"].str.startswith('TECNICO'), 
]

valores = ["EQUIPAMENTO"]*2 + ["MÃO DE OBRA_CURSO"] + ["MÃO DE OBRA"]*40

df_plan["CATEGORIA INSUMO"] = np.select(condicoes, valores, default="MATERIAL")

In [ ]:
df_plan

### Avaliação dos resultados

In [ ]:
df_plan

In [ ]:
df_plan[df_plan["CATEGORIA INSUMO"] == "MÃO DE OBRA"]